In [3]:
bucket_path = "gs://flight-analysis-ms-bucket/cleaned/cleaned_flight_data.parquet"

# Test reading from GCS
try:
    print("Attempting to access the GCS bucket...")
    spark.read.parquet(bucket_path).show(5)
    print("Success! Data loaded from GCS.")
except Exception as e:
    print(f"Error accessing GCS: {e}")


Attempting to access the GCS bucket...


+-------------------+-----------------+------+----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+----+-------+-----+----------+---------+--------+
|         FlightDate|          Airline|Origin|Dest|Cancelled|Diverted|DepDel15|ArrivalDelayGroups|DistanceGroup|OriginState|DestState|Operating_Airline|Year|Quarter|Month|DayofMonth|DayOfWeek|Distance|
+-------------------+-----------------+------+----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+----+-------+-----+----------+---------+--------+
|2018-01-23 00:00:00|Endeavor Air Inc.|   ABY| ATL|        0|       0|       0|                -1|            1|         GA|       GA|               9E|2018|      1|    1|        23|        2|     145|
|2018-01-24 00:00:00|Endeavor Air Inc.|   ABY| ATL|        0|       0|       0|                -1|            1|         GA|       GA|               9E|2018|      1|    1|        24|        3|

In [4]:
# Define the file path
file_path = "gs://flight-analysis-ms-bucket/cleaned/cleaned_flight_data.parquet"

# Load the Parquet file into a Spark DataFrame
flight_data = spark.read.parquet(file_path)

# Show the schema to confirm the structure of the data
flight_data.printSchema()

# Display a sample of the data
flight_data.show(5)


root
 |-- FlightDate: timestamp_ntz (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: byte (nullable = true)
 |-- Diverted: byte (nullable = true)
 |-- DepDel15: byte (nullable = true)
 |-- ArrivalDelayGroups: byte (nullable = true)
 |-- DistanceGroup: byte (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- DestState: string (nullable = true)
 |-- Operating_Airline: string (nullable = true)
 |-- Year: short (nullable = true)
 |-- Quarter: byte (nullable = true)
 |-- Month: byte (nullable = true)
 |-- DayofMonth: byte (nullable = true)
 |-- DayOfWeek: byte (nullable = true)
 |-- Distance: integer (nullable = true)

+-------------------+-----------------+------+----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+----+-------+-----+----------+---------+--------+
|         FlightDate|          Airline|Origin|Dest|Cancelled|Div

In [5]:
print(f"Total rows: {flight_data.count()}")
print(f"Total columns: {len(flight_data.columns)}")


Total rows: 5689512
Total columns: 18


In [8]:
from pyspark.sql.functions import col


# Count nulls per column
flight_data.select([col(c).isNull().alias(c) for c in flight_data.columns]).show()


+----------+-------+------+-----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+-----+-------+-----+----------+---------+--------+
|FlightDate|Airline|Origin| Dest|Cancelled|Diverted|DepDel15|ArrivalDelayGroups|DistanceGroup|OriginState|DestState|Operating_Airline| Year|Quarter|Month|DayofMonth|DayOfWeek|Distance|
+----------+-------+------+-----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+-----+-------+-----+----------+---------+--------+
|     false|  false| false|false|    false|   false|   false|             false|        false|      false|    false|            false|false|  false|false|     false|    false|   false|
|     false|  false| false|false|    false|   false|   false|             false|        false|      false|    false|            false|false|  false|false|     false|    false|   false|
|     false|  false| false|false|    false|   false|   false|             f

In [9]:
# Summing null values for each column
null_counts = flight_data.select([col(c).isNull().cast("int").alias(c) for c in flight_data.columns]) \
                         .agg(*[sum(col(c)).alias(c) for c in flight_data.columns])

# Show total null counts per column
null_counts.show()


PySparkTypeError: [NOT_ITERABLE] Column is not iterable.

In [10]:
from pyspark.sql.functions import sum

# Summing null values for each column
null_counts = flight_data.select([col(c).isNull().cast("int").alias(c) for c in flight_data.columns]) \
                         .agg(*[sum(col(c)).alias(c) for c in flight_data.columns])

# Show total null counts per column
null_counts.show()


+----------+-------+------+----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+----+-------+-----+----------+---------+--------+
|FlightDate|Airline|Origin|Dest|Cancelled|Diverted|DepDel15|ArrivalDelayGroups|DistanceGroup|OriginState|DestState|Operating_Airline|Year|Quarter|Month|DayofMonth|DayOfWeek|Distance|
+----------+-------+------+----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+----+-------+-----+----------+---------+--------+
|         0|      0|     0|   0|        0|       0|   86575|                 0|            0|          0|        0|                0|   0|      0|    0|         0|        0|       0|
+----------+-------+------+----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+----+-------+-----+----------+---------+--------+



In [11]:
# Impute missing values in DepDel15 with 0
flight_data = flight_data.fillna({"DepDel15": 0})

# Verify that there are no more nulls
flight_data.select([sum(col(c).isNull().cast("int")).alias(c) for c in flight_data.columns]).show()


+----------+-------+------+----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+----+-------+-----+----------+---------+--------+
|FlightDate|Airline|Origin|Dest|Cancelled|Diverted|DepDel15|ArrivalDelayGroups|DistanceGroup|OriginState|DestState|Operating_Airline|Year|Quarter|Month|DayofMonth|DayOfWeek|Distance|
+----------+-------+------+----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+----+-------+-----+----------+---------+--------+
|         0|      0|     0|   0|        0|       0|       0|                 0|            0|          0|        0|                0|   0|      0|    0|         0|        0|       0|
+----------+-------+------+----+---------+--------+--------+------------------+-------------+-----------+---------+-----------------+----+-------+-----+----------+---------+--------+



In [12]:
# Define the path to save the intermediate dataset
intermediate_path = "gs://flight-analysis-ms-bucket/trusted/handled_missing_values.parquet"

# Save the current dataset
flight_data.write.mode("overwrite").parquet(intermediate_path)

print(f"Data with missing values handled saved to {intermediate_path}")


Data with missing values handled saved to gs://flight-analysis-ms-bucket/trusted/handled_missing_values.parquet
